In [1]:
import pandas as pd
import numpy as np
import datarobot as dr
from collections import defaultdict
import datetime as dt 
from helpers import make_cv_data, get_top_of_leaderboard

In [2]:
pd.set_option('display.max_columns', 58, 'display.max_rows',200)
dr.Client(config_path='/Users/chandler.mccann/DataRobot/drconfig.yaml')

In [66]:
dataset= dr.Dataset.get('63d88627ae5a7849ff91e801') #updated https://catalog.waterpointdata.org/intermediate/all/training/data/all_training_water_points.csv
dtmp = dr.Dataset.get_as_dataframe(dataset)
dtmp

,wpdx_id,status_id,clean_country_id,clean_adm1,clean_adm2,water_tech_clean,water_source_clean,water_tech_category,water_source_category,management_clean,pay_clean,subjective_quality_clean,report_date,unified_install_year,unified_installer,age_in_years,distance_to_primary,distance_to_secondary,distance_to_tertiary,distance_to_city,distance_to_town,is_urban,precipitation_5year,precipitation_10year,landcover_code,acled_index,isobioclimates,bgs_dtw,bgs_prod,...,bgs_recharge,water_risk,rwi,assigned_population,local_population,usage_cap,pressure,crucialness,population_1km,men_ratio_1km,women_of_reproductive_age_ratio_1km,youth_ratio_1km,elderly_ratio_1km,children_under_five_ratio_1km,population_10km,men_ratio_10km,women_of_reproductive_age_ratio_10km,youth_ratio_10km,elderly_ratio_10km,children_under_five_ratio_10km,population_100km,men_ratio_100km,women_of_reproductive_age_ratio_100km,youth_ratio_100km,elderly_ratio_100km,children_under_five_ratio_100km,orig_status_id,years_since_report,fold
0,6G6HRQVX+884,No,TZA,Tabora,Kaliua,Hand Pump - SWN 80,Protected Hand Dug Well,Hand Pump,Well,Community Management,NaN,Acceptable quality,1978-10-27,2005.0,TDFT,-26.182605,14013.476487,8912.711524,7274.314685,113170.415488,10773.294051,False,930.227121,972.694238,10,0.0,35.0,7.0,0.5,...,76.916908,2.135211,-0.380,1141.0,1141.0,300.0,3.803333,1.000000,1.131023e+03,0.484749,0.222700,0.190494,0.039612,0.200132,17956.166016,0.485076,0.222559,0.190489,0.039615,0.200130,9.875759e+05,0.499021,0.216461,0.189032,0.044001,0.195434,Yes,-26.182605,1
1,6CVG7H77+X76,No,LBR,Bong,Zota,Hand Pump - Afridev,Protected Hand Dug Well,Hand Pump,Well,Community Management,NaN,Acceptable quality,1980-01-06,2001.0,ACF (Action Contre la Faim) acf,-20.988795,317.614248,643.774615,18120.449960,87410.305606,457.381800,False,NaN,NaN,40,0.0,37.0,0.0,0.5,...,184.720947,1.913147,-0.349,265.0,2641.0,300.0,0.883333,0.100341,2.623720e+03,0.491008,0.245504,0.176352,0.061836,0.181188,10136.115234,0.490954,0.245530,0.176353,0.061836,0.181188,1.358279e+06,0.491954,0.237857,0.179004,0.063492,0.174644,Yes,-20.988795,3
2,6CVG7H77+H7J,Yes,LBR,Bong,Zota,Hand Pump - Afridev,Protected Hand Dug Well,Hand Pump,Well,Community Management,NaN,Bad color,1980-01-06,2006.0,lds,-25.988213,234.766905,552.920277,18224.666718,87475.907446,363.222146,False,NaN,NaN,40,0.0,37.0,0.0,0.5,...,184.720947,1.913147,-0.349,310.0,2641.0,300.0,1.033333,0.117380,2.641448e+03,0.491008,0.245504,0.176352,0.061836,0.181188,10136.115234,0.490954,0.245530,0.176353,0.061836,0.181188,1.358279e+06,0.491954,0.237857,0.179004,0.063492,0.174644,Yes,-25.988213,4
3,6CPJJ4QH+4Q9,Yes,LBR,Grand Kru,Lower Kru Coast,Hand Pump - Afridev,Protected Hand Dug Well,Hand Pump,Well,Community Management,NaN,Acceptable quality,1980-01-06,2006.0,ERS,-25.988213,31311.148028,23133.296783,52.447044,137011.072264,22522.916950,False,NaN,NaN,10,0.0,37.0,7.0,0.5,...,166.321411,2.002818,-0.190,93.0,690.0,300.0,0.310000,0.134783,6.903995e+02,0.512256,0.227677,0.200803,0.047569,0.164559,4280.487305,0.521022,0.224120,0.204645,0.046424,0.161919,5.666357e+05,0.530955,0.221888,0.210188,0.043811,0.152914,Yes,-25.988213,0
4,6CPJJ4VC+7XG,No,LBR,Grand Kru,Lower Kru Coast,Hand Pump - Afridev,Protected Hand Dug Well,Hand Pump,Well,Community Management,NaN,NaN,1980-01-06,2005.0,ERS,-24.988877,32282.995761,23192.509337,34.385195,137730.102857,23426.773635,False,NaN,NaN,10,0.0,37.0,7.0,0.5,...,166.321411,2.002818,-0.190,38.0,325.0,300.0,0.126667,0.116923,2.510544e+02,0.512256,0.227677,0.200803,0.047569,0.164559,3986.021729,0.518438,0.224983,0.202200,0.047136,0.163631,5.666357e+05,0.530955,0.221888,0.210188,0.043811,0.152914,Yes,-24.988877,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308330,7G2P8J6M+3WH,Yes,ETH,Benishangul Gumz,Assosa,Mechanized Pump,NaN,Mechanized Pump,NaN,NaN

In [ ]:
dtmp['fold']=dtmp['fold'].apply(lambda x: 'fold_'+str(x))
dtmp.fold.unique()

array(['fold_1', 'fold_3', 'fold_4', 'fold_0', 'fold_2', 'fold_-1'],
      dtype=object)

In [ ]:
dtmp.groupby('fold')['years_since_report'].describe()

,count,mean,std,min,25%,50%,75%,max
fold,,,,,,,,
fold_-1,46250.0,5.352889,8.514219,-22.924495,0.090351,1.366216,8.635359,121.111316
fold_0,53013.0,10.136232,11.101931,-26.223673,2.491495,6.365634,14.406867,115.178272
fold_1,52364.0,10.176823,11.121888,-26.182605,2.543516,6.384799,14.571141,115.378139
fold_2,52396.0,10.172768,11.072378,-14.998255,2.532564,6.384799,14.538286,117.201585
fold_3,52481.0,10.159977,11.059500,-29.621416,2.502447,6.382061,14.568403,114.266549
fold_4,51831.0,10.158406,11.049134,-29.988295,2.513399,6.379323,14.473945,114.266549


# Global model approach
1) Update the individual countries (UGA, SLE, NGA, GHA, TZA)to have a fold that is the most recent 15% of data.
2) Build global model
    - take most recent 20% of data from the non-individual countries
    - add a column called country2 that is name of the original country or 'holdout' if it's in the 20% most recent record
    - add all the data from UGA/SLE/NGA/ GHA/TZA to the ^ data (dont use the countries for holdout)
    - run 'partition feature' with coutnry 2 as the column and the holdout value as the holdout set
    

# Global Model

In [ ]:
# #Create global model training data
# #extract all data from single model countries
# single_countries = [ 'UGA','SLE', 'NGA', 'GHA', 'TZA']
# glob_train= df.loc[df.clean_country_id.isin(single_countries)]

# #combine data from all other countries
# glob_countries= [c for c in df.clean_country_id.unique() if c not in single_countries]
# glob_temp = df.loc[df.clean_country_id.isin(glob_countries)]

# #create dataset for global model countries with holdout fold of most recent data
# temp=pd.DataFrame()
# for ctry in glob_countries:
#     data = make_cv_data(glob_temp, ctry) # still using 15 % holdout
#     temp = pd.concat([temp, data], ignore_index=True, sort=False)
    

# define feature list

In [ ]:
feats = [
# 'wpdx_id',
 'status_id',
 #'clean_country_id',
 #'clean_adm1',
 #'clean_adm2',
 'water_tech_clean',
 'water_source_clean',
 'water_tech_category',
 'water_source_category',
 'management_clean',
 #'report_date',
 #'unified_install_year',
 'unified_installer',
 'age_in_years',
 'distance_to_primary',
 'distance_to_secondary',
 'distance_to_tertiary',
 'distance_to_city',
 'distance_to_town',
 'is_urban',
 'precipitation_5year',
 'precipitation_10year',
 'landcover_code',
 'acled_index',
 'isobioclimates',
 'bgs_dtw',
 'bgs_prod',
 'bgs_stor',
 'bgs_recharge',
 'water_risk',
 'rwi',
 'assigned_population',
 'local_population',
 'usage_cap',
 'pressure',
 #'criticality',
'crucialness',
 'population_1km',
 'men_ratio_1km',
 'women_of_reproductive_age_ratio_1km',
 'youth_ratio_1km',
 'elderly_ratio_1km',
 'children_under_five_ratio_1km',
 'population_10km',
 'men_ratio_10km',
 'women_of_reproductive_age_ratio_10km',
 'youth_ratio_10km',
 'elderly_ratio_10km',
 'children_under_five_ratio_10km',
 'population_100km',
 'men_ratio_100km',
 'women_of_reproductive_age_ratio_100km',
 'youth_ratio_100km',
 'elderly_ratio_100km',
 'children_under_five_ratio_100km',
 'orig_status_id',
 'years_since_report',
# 'fold',
# 'report_year'
]
print(len(feats))

50


# define variables to change to categorical ints, and partitioning method, bias and fairness testin

In [ ]:
VAR_TYPES = {'acled_index'       : 'categoricalInt',
             'isobioclimates'  : 'categoricalInt',
             'landcover_code'         : 'categoricalInt',
             }


In [ ]:
#define partitioning

spec = dr.UserCV('fold', 'fold_-1', seed=0)

#set Bias and Fairness Testing
advanced_ops=dr.AdvancedOptions(
    protected_features=['clean_adm1'],
    preferable_target_value='No',
    fairness_metrics_set='favorableAndUnfavorablePredictiveValueParity',
    fairness_threshold='0.8')
#global model - update protected feature to country
advanced_ops2=dr.AdvancedOptions(
    protected_features=['clean_country_id'],
    preferable_target_value='No',
    fairness_metrics_set='favorableAndUnfavorablePredictiveValueParity',
    fairness_threshold='0.8')

# Single country model
Loop through each country, create and story projects in a dictionary by country

In [ ]:
#single country model
today = dt.datetime.today().strftime('%Y-%m-%d')
project_dict = defaultdict()
for country in [ 'SLE', 'NGA', 'GHA', 'TZA']:
    #def create_single_country(df, country):
    #dframe= make_cv_data(df,country)
    dframe = dtmp.loc[dtmp.clean_country_id==country]
    print('buiding dataset for ' , country, 'with shape ', dframe.shape)
    print('--------------------------------------------------')
    print('Building models for: {}'.format(country))
    print('--------------------------------------------------')

    # create feature list
    project_name = f"{country}_{today}"
    project = dr.Project.create(dframe, project_name=project_name)

    # create feature list
    project.create_featurelist('feats',feats) # use our defined feature list
    #ceate variable type transforms from int to cat
    print('Starting variable type transformations.')

    # now we'll run through the dict and do the transforms one-by-one
    #this is very verbose...
    new_features={}
    for feat, transf in VAR_TYPES.items():
        new_name = feat + ' (' + transf + ')'
        try:
            new_features[feat] = project.create_type_transform_feature(name=new_name,
                                                                    parent_name=feat,
                                                                    variable_type=transf).name
            print('Transformed', feat, 'to', new_name, 'successfully.')
        except dr.errors.ClientError as e:
            assert e.status_code == 422
            print(feat, 'transformation to', transf, 'failed.')

    featurelists = project.get_featurelists()
    new_featurelists = {}

    print('Adding feature lists.')
    for fl in featurelists:
        new_featlist_name = fl.name + ' (retyped)'
        new_featurelists[new_featlist_name] = project.create_featurelist(
            name=new_featlist_name,
            features=[new_features[f] if f in new_features.keys() else f for f in fl.features] #this wont remove old. Need to set original features out
        ).id


    # start project with 2 backtests and BF testing
    project.analyze_and_model(
        target='status_id',
        mode=dr.enums.AUTOPILOT_MODE.FULL_AUTO,
        advanced_options=advanced_ops,
        partitioning_method= spec,
        featurelist_id=new_featurelists['feats (retyped)'],
        worker_count=-1,
    )
    project_dict[country]= project
    print(project.project_name, project.get_uri())

    
    

4426
buiding dataset for  SLE with shape  (29506, 60)
--------------------------------------------------
Building models for: SLE
--------------------------------------------------
Starting variable type transformations.
Transformed acled_index to acled_index (categoricalInt) successfully.
Transformed isobioclimates to isobioclimates (categoricalInt) successfully.
Transformed landcover_code to landcover_code (categoricalInt) successfully.
Adding feature lists.
SLE_2023-01-31 https://app.datarobot.com/projects/63d92a316ebf7c6076da0739/models
7261
buiding dataset for  NGA with shape  (48405, 60)
--------------------------------------------------
Building models for: NGA
--------------------------------------------------
Starting variable type transformations.
Transformed acled_index to acled_index (categoricalInt) successfully.
Transformed isobioclimates to isobioclimates (categoricalInt) successfully.
Transformed landcover_code to landcover_code (categoricalInt) successfully.
Adding fea

# Global Model

In [ ]:
# #global model
# # create feature list
# project_name = f"{'global_model'}_{today}"
# project = dr.Project.create(glob_mod_final_train, project_name=project_name) #update<---------

# # create feature list
# project.create_featurelist('feats',feats) # use our defined feature list
# #ceate variable type transforms from int to cat
# print('Starting variable type transformations.')

# # now we'll run through the dict and do the transforms one-by-one
# #this is very verbose...
# new_features={}
# for feat, transf in VAR_TYPES.items():
#     new_name = feat + ' (' + transf + ')'
#     try:
#         new_features[feat] = project.create_type_transform_feature(name=new_name,
#                                                                 parent_name=feat,
#                                                                 variable_type=transf).name
#         print('Transformed', feat, 'to', new_name, 'successfully.')
#     except dr.errors.ClientError as e:
#         assert e.status_code == 422
#         print(feat, 'transformation to', transf, 'failed.')

# featurelists = project.get_featurelists()
# new_featurelists = {}

# print('Adding feature lists.')
# for fl in featurelists:
#     new_featlist_name = fl.name + ' (retyped)'
#     new_featurelists[new_featlist_name] = project.create_featurelist(
#         name=new_featlist_name,
#         features=[new_features[f] if f in new_features.keys() else f for f in fl.features] #this wont remove old. Need to set original features out
#     ).id


# # start project with 2 backtests and BF testing
# project.analyze_and_model(
#     target='status_id',
#     mode=dr.enums.AUTOPILOT_MODE.FULL_AUTO,
#     advanced_options=advanced_ops2,
#     partitioning_method= spec,
#     featurelist_id=new_featurelists['feats (retyped)'],
#     worker_count=-1,
# )
# project_dict['GLOBAL']= project
# print(project.project_name, project.get_uri())

Starting variable type transformations.
Transformed acled_index to acled_index (categoricalInt) successfully.
Transformed isobioclimates to isobioclimates (categoricalInt) successfully.
Transformed landcover_code to landcover_code (categoricalInt) successfully.
Adding feature lists.
global_model_2023-01-31 https://app.datarobot.com/projects/63d92d0129be85f20ada082a/models


# Feature Reduction 
Funtion to iteratively retrain and eliminate the least predictive feature, and show results in a dataframe. Defaults to stop at 10 feautures. 

In [39]:

def leave_one_out(project_id, model_id, n_features=10):
    models = []
    m = dr.Model.get(project_id, model_id)
    p=dr.Project.get(project_id)
    p.wait_for_autopilot()
    fi = pd.DataFrame(m.get_or_request_feature_impact())
    while fi.shape[0] > n_features:
        try:
            n = fi.shape[0] - 1
            print('{} features'.format(n))
            fi = fi.head(n)
            new_fl = p.create_featurelist('M{} top {}'.format(m.model_number, n),
                                        fi['featureName'].tolist())
            mj = m.retrain(featurelist_id = new_fl.id)
            m = mj.get_result_when_complete()
            m = dr.Model.get(m.project_id, m.id)
            m.cross_validate()
            models.append(m)
            fi = pd.DataFrame(m.get_or_request_feature_impact())
        except:
            continue
    return models, fi #list of models, dataframe


def analyze_leave_one_out(models_list):
    data_list = []
    for m in models_list:
        m = dr.Model.get(m.project_id, m.id)
        results = pd.DataFrame(data={'featurelist_name': m.featurelist_name,
                                     'feature_list_id': m.featurelist_id,
                                    'crossValidation_logloss': m.metrics['LogLoss']['crossValidation'],
                                    'crossValidation_auc': m.metrics['AUC']['crossValidation'],
                                    'val_logloss': m.metrics['LogLoss']['validation'],
                                    'val_auc': m.metrics['AUC']['validation'],
                                    'holdout_logloss':m.metrics['LogLoss']['holdout'],
                                    'holdout_auc':m.metrics['AUC']['holdout'],
                                    'model_id': m.id,
                                    'project_id': m.project_id}, index=[0])
        data_list.append(results)
        data = pd.concat(data_list)
        data['n_features'] = data['featurelist_name'].str.split().str[-1].astype(int)
        data.sort_values(by='holdout_auc', ascending=False)
    return data.round(3) #dataframe

In [37]:
def models_from_reduced_fl(project, search="top"):
    """Return list of models from feature reduction process"""
    reduced_feat_lists = [f.name for f in project.get_featurelists() if search in f.name]
    models = project.get_models()
    model_list = [m for m in models if m.featurelist.name in reduced_feat_lists]
    
    return model_list

In [42]:
def reduced_fl_champ(df):
    """return feature list with fewest features
      where the holdout auc and cross val AUC are within 95% of the max from all runs"""
    
    m1='holdout_auc'
    m2='crossValidation_auc'
    out = df.loc[(df[m1]>=(df[m1].max()*.95))&(df[m2]>=df[m2].max()*.95)].sort_values(by = 'n_features', ascending = True)
    
    return out.iloc[0]["feature_list_id"]

In [105]:
#this needs work ############


def reduced_model_champ(df):
    """return feature list with fewest features
      where the holdout auc and cross val AUC are within 95% of the max from all runs"""
    
    m1='holdout_auc'
    m2='crossValidation_auc'
    start =.99
    for i in [0,.01,.02,.03,.04]:
        try:
            val=start -i
            out = df.loc[(df[m1]>=(df[m1].max()*val))&(df[m2]>=df[m2].max()*val)].sort_values(by = m1, ascending = True)
        except:
            continue
    print('model performant within the ', start, 'percentile of holdout and cross-validation AUC')
    
    return out.iloc[0]["model_id"]

# get best model from 1st round of autopilot

In [3]:
glob_proj = dr.Project.get('63d92d0129be85f20ada082a')
tza_proj = dr.Project.get('63d92c52fba1f19a2dda0965')
nga_proj = dr.Project.get('63d92ae329be85f20ada0800')
gha_proj = dr.Project.get('63d92ba2819f068003da011c')

### best non-frozen model from first run on CV

In [ ]:
glob_m1= get_top_of_leaderboard(glob_proj).iloc[1]["model_id"] #best non-frozen model from first run on CV

### run leave one out, get the results of the different featurelists

In [ ]:
glob_list, glob_fi = leave_one_out(glob_proj.id, glob_m1)
glob_res = analyze_leave_one_out(glob_list)
glob_res

In progress: 0, queued: 0 (waited: 0s)
48 features
47 features
46 features
45 features


### find the best feature list by holdout and AUC scores (within 95% of top model)

In [ ]:
glob_fl =reduced_fl_champ(glob_res)
glob_fl

'63d9faa64c03aa5eae797879'

### run comprehensive mode on that feature list

In [ ]:
glob_proj.start_autopilot(glob_fl,mode=dr.enums.AUTOPILOT_MODE.COMPREHENSIVE)

In [ ]:
glob_proj.wait_for_autopilot()

In [ ]:
#pick up work where we left off
#glob_proj = dr.Project.get('63d92d0129be85f20ada082a')


### get the models from the best feature list run

In [ ]:
gl_list= models_from_reduced_fl(glob_proj, "M181 top 37")
gl_res = analyze_leave_one_out(gl_list)
gl_res.head()

,featurelist_name,feature_list_id,crossValidation_logloss,crossValidation_auc,val_logloss,val_auc,holdout_logloss,holdout_auc,model_id,project_id,n_features
0,M181 top 37,63d9faa64c03aa5eae797879,0.380586,0.845194,0.382,0.846,0.186,0.916,63dafc7e95a7166db24c2699,63d92d0129be85f20ada082a,37
0,M181 top 37,63d9faa64c03aa5eae797879,0.380054,0.845708,0.382,0.845,0.267,0.809,63daea342c9b86c36134d29c,63d92d0129be85f20ada082a,37
0,M181 top 37,63d9faa64c03aa5eae797879,0.380034,0.84569,0.382,0.845,0.272,0.798,63da6648d9f501ae90b0164b,63d92d0129be85f20ada082a,37
0,M181 top 37,63d9faa64c03aa5eae797879,0.38044,0.845414,0.383,0.844,0.270,0.800,63db102ac0d202a79fc8a736,63d92d0129be85f20ada082a,37
0,M181 top 37,63d9faa64c03aa5eae797879,0.380492,0.845368,0.383,0.844,0.270,0.800,63db1029c0d202a79fc8a731,63d92d0129be85f20ada082a,37


### Pick the best model
There is a way to do it but we should really consider the algorithm family. I have manually selected them for now

# Nigeria

In [63]:
nga_proj = dr.Project.get('63d92ae329be85f20ada0800')

In [76]:
#find which feature list performed the best from reduction
nga_list= models_from_reduced_fl(nga_proj)#, "M181 top 37")
nga_res = analyze_leave_one_out(nga_list)
nga_res.head()

,featurelist_name,feature_list_id,crossValidation_logloss,crossValidation_auc,val_logloss,val_auc,holdout_logloss,holdout_auc,model_id,project_id,n_features
0,M56 top 48,63d9eb852fa26f9ed279791a,0.459228,0.834886,0.458,0.838,0.525,0.815,63dad9d3086d6c45cd8145ab,63d92ae329be85f20ada0800,48
0,M56 top 48,63d9eb852fa26f9ed279791a,0.459108,0.835754,0.459,0.838,0.627,0.731,63dad9770e600b89a7b9dbdd,63d92ae329be85f20ada0800,48
0,M56 top 48,63d9eb852fa26f9ed279791a,0.45982,0.834138,0.461,0.836,0.619,0.731,63d9eb864e0140db1891e56a,63d92ae329be85f20ada0800,48
0,M121 top 46,63da489c7f98fe55465f5b0d,0.460216,0.834136,0.463,0.835,0.616,0.733,63da489d3f8d13343a797bf1,63d92ae329be85f20ada0800,46
0,M174 top 42,63d9ef3175bc87a4cc797884,0.461118,0.83312,0.463,0.834,0.621,0.729,63d9ef322669625bc15f57f8,63d92ae329be85f20ada0800,42


In [89]:
nga_top10 = nga_res[nga_res.feature_list_id=='63da019402cf20380391e84b'] #top 10 feature list
nga_top10.sort_values(by='holdout_auc', ascending = False)

,featurelist_name,feature_list_id,crossValidation_logloss,crossValidation_auc,val_logloss,val_auc,holdout_logloss,holdout_auc,model_id,project_id,n_features
0,M471 top 10,63da019402cf20380391e84b,0.516138,0.778906,0.521,0.775,0.623,0.721,63daaef2acf8b445a3ab6ea1,63d92ae329be85f20ada0800,10
0,M471 top 10,63da019402cf20380391e84b,0.510114,0.78614,0.516,0.781,0.623,0.721,63daaef2acf8b445a3ab6e97,63d92ae329be85f20ada0800,10
0,M471 top 10,63da019402cf20380391e84b,0.530496,0.757996,0.535,0.754,0.626,0.720,63daaef2acf8b445a3ab6e9b,63d92ae329be85f20ada0800,10
0,M471 top 10,63da019402cf20380391e84b,0.513124,0.782356,0.515,0.781,0.626,0.720,63daaef3acf8b445a3ab6ea3,63d92ae329be85f20ada0800,10
0,M471 top 10,63da019402cf20380391e84b,0.507284,0.79053,0.511,0.788,0.623,0.719,63daaef3acf8b445a3ab6ec5,63d92ae329be85f20ada0800,10
0,M471 top 10,63da019402cf20380391e84b,None,None,0.523,0.773,0.625,0.719,63daaef2acf8b445a3ab6e90,63d92ae329be85f20ada0800,10
0,M471 top 10,63da019402cf20380391e84b,None,None,0.516,0.781,0.625,0.719,63daaef3acf8b445a3ab6eaf,63d92ae329be85f20ada0800,10
0,M471 top 10,63da019402cf20380391e84b,None,None,0.537,0.751,0.629,0.718,63daaef3acf8b445a3ab6ebf,63d92ae329be85f20ada0800,10
0,M471 top 10,63da019402cf20380391e84b,None,None,0.542,0.746,0.636,0.716,63daaef2acf8b445a3ab6e96,63d92ae329be85f20ada0800,10
0,M471 top 10,63da019402cf20380391e84b,None,None,0.541,0.752,0.641,0.716,63daaef3acf8b445a3ab6eb4,63d92ae329be85f20ada0800,10


In [106]:
reduced_model_champ(nga_top10)

model performant within the  0.99 percentile of holdout and cross-validation AUC


'63daaef2acf8b445a3ab6e9e'

In [107]:
nga_proj.start_prepare_model_for_deployment('63daaef3acf8b445a3ab6ea3')

In [117]:
nga_to_deploy = dr.Model.get(nga_proj.id,'63daaef3acf8b445a3ab6ea3')
nga_to_deploy.blueprint.model_type

'Nystroem Kernel SVM Classifier'

In [118]:
nga_proj.get_models(search_params={'sample_pct':100,'name':nga_to_deploy.blueprint.model_type})

[]